# Obtención de los datos

Se colocó de forma manual cada receta del libro en un fichero de texto plano con un nombre único para cada una.

Ventajas:
- Dar un identificador único a cada receta.
- Tener cada receta solo con el texto relacionado a ella, dígase ingredientes, consejos generales y pasos.
- En el futuro si se tienen nuevas recetas solo es necesario añadirlas a la colección.

Puede verse este conjunto de ficheros de texto como el libro en una forma mejor estructurada para el procesamiento individual de cada receta.

A partir de aquí se desea crear un proceso capaz de identificar el tipo y cantidad de cada ingrediente involucrado para cada receta.

In [1]:
from pathlib import Path

import spacy
from sqlmodel import SQLModel, Session, create_engine

from src.parsers import RecipesData, RecipeParser

## Obtener texto de cada receta

Para obtener el texto de cada receta se puede crear una instancia de la clase `RecipesData` y trabajar con ella de forma similar a un `dict`. Esta clase va a tener mapeado el nombre de cada fichero txt al texto ya parseado de la receta que contiene, esta información se maneja dentro de una instancia de `RecipeParser`.

Se quiere extraer de cada receta:
- nombre de la receta
- recetas de las que depende
- ingredientes
- cantidad de raciones

In [2]:
data = RecipesData(Path("files/recipes/"))
data

Recipes for 'salsas': 13
Recipes for 'ensalada y aliños': 24
Recipes for 'pescados': 21
Recipes for 'aves': 23
Recipes for 'postres': 39
Recipes for 'sopas': 24
Recipes for 'cerales (arroz y maiz)': 27
Recipes for 'viandas, hortalizas y otros vegetales': 37
Recipes for 'carne de res': 26


RecipesData(recipes=234, empty=82, total=316)

In [13]:
data["arroz_lentejas"]

Arroz con lentejas (8)
id=None text='½ libra de lentejas' amount=0.0 measure='' food='' verified=False section=None recipe_id='arroz_lentejas'
id=None text='1 libra de arroz' amount=0.0 measure='' food='' verified=False section=None recipe_id='arroz_lentejas'
id=None text='4 tazas de agua' amount=0.0 measure='' food='' verified=False section=None recipe_id='arroz_lentejas'
id=None text='2 ajíes' amount=0.0 measure='' food='' verified=False section=None recipe_id='arroz_lentejas'
id=None text='¼ libra de tocino o beicon' amount=0.0 measure='' food='' verified=False section=None recipe_id='arroz_lentejas'
id=None text='¼ taza de aceite' amount=0.0 measure='' food='' verified=False section=None recipe_id='arroz_lentejas'
id=None text='½ libra de cebollas' amount=0.0 measure='' food='' verified=False section=None recipe_id='arroz_lentejas'
id=None text='3 dientes de ajo' amount=0.0 measure='' food='' verified=False section=None recipe_id='arroz_lentejas'
id=None text='1 hoja de laurel' amo

## Añadir las recetas a la BD

El siguiente paso es añadir todas las recetas a la BD para luego poder modelar las relaciones de dependencia entre ellas.


In [14]:
DB = "files/database.db"
ENGINE = create_engine(f"sqlite:///{DB}")

if not Path(DB).exists():
    SQLModel.metadata.create_all(ENGINE)


> Las recetas parseadas son accesible en el atributo `recipes` de la clase `RecipesData`.

In [15]:
with Session(ENGINE) as session:
    for recipe in data.recipes:
        recipe.save_to_db(session)
    
    session.commit()

## Reconocimiento de entidades en los ingredientes

Para obtener la información necesaria de los ingredientes en cada receta se emplea reconocimiento de entidades en texto. Para ello se anota un número importante de ingredientes en cuanto a los _tags_: `AMOUNT`, `MEASURE` y `FOOD`.

> Para realizar las anotaciones fue empleado [Label Studio](https://labelstud.io/).

Por ejemplo del ingrediente "1 libra de arroz" se deben obtener las entidades:

- "1": `AMOUNT`
- "libra": `MEASURE`
- "arroz": `FOOD`

Los datos anotados se emplearon para entrenar un modelo que se adapte al problema.

In [41]:
# with Path("files/ner-training.txt").open(mode="w", encoding="utf8") as f:
#     for recipe in data.recipes:
#         for ingredient in recipe.ingredients:
#             f.write(f"{ingredient.text}\n") 

In [8]:
nlp = spacy.load(Path("files/nlp_model"))

In [10]:
for label in nlp.get_pipe("ner").labels:
    print(label, "-", spacy.explain(label))

AMOUNT - None
FOOD - None
LOC - Non-GPE locations, mountain ranges, bodies of water
MEASURE - None
MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art
ORG - Companies, agencies, institutions, etc.
PER - Named person or family.


In [11]:
from spacy import displacy
for ing in data["arroz_frito"].ingredients:
    doc = nlp(ing.text)
    displacy.serve(doc)

    # print(
    #     [(e, e.label_) for e in doc.ents], [
    #     f"{token.text}:{token.lemma_}"
    #     if token.text != token.lemma_
    #     else token.text
    #     for token in doc
    # ])

/home/verdesia/5to/PD/nitza/env/lib/python3.9/site-packages/spacy/displacy/__init__.py:98: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



In [10]:
doc = nlp("¼ de taza de vino seco")
[(e, e.label_) for e in doc.ents]

[(¼, 'MEASURE'), (taza de vino, 'FOOD')]